In [1]:
import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64

from matplotlib.path import Path
import matplotlib.patches as patches

In [51]:
def strokes_to_lines(strokes):
    """
    Convert stroke-3 format to polyline format.
    List contains sublist of continuous line segments (strokes).    
    """
    x = 0
    y = 0
    lines = []
    line = []
    for i in range(len(strokes)):
        if strokes[i, 2] == 1:
            x += float(strokes[i, 0])
            y += float(strokes[i, 1])
            line.append([x, y])
            lines.append(line)
            line = []
        else:
            x += float(strokes[i, 0])
            y += float(strokes[i, 1])
            line.append([x, y])
    return lines

def polyline_pathmaker(lines):
    x = []
    y = []

    codes = [Path.MOVETO] # start with moveto command always
    for i,l in enumerate(lines):
        for _i,_l in enumerate(l):
            x.append(_l[0])
            y.append(_l[1])
            if _i<len(l)-1:
                codes.append(Path.LINETO) # keep pen on page
            else:
                if i != len(lines)-1: # final vertex
                    codes.append(Path.MOVETO)
    verts = zip(x,y)            
    return verts, codes

def path_renderer(verts, codes, save_path):
    fig = plt.figure(figsize=(6,6))    
    ax = plt.subplot(111)    
    padding = 10
    xlb = min([x for (x,y) in verts]) - padding
    xub = max([x for (x,y) in verts]) + padding
    ylb = min([y for (x,y) in verts]) - padding
    yub = max([y for (x,y) in verts]) + padding
    lb = min(xlb,ylb)
    ub = max(xub,yub)
    ax.axis('off')
    
    ax.set_xlim(lb,ub)
    ax.set_ylim(lb,ub)     
    
    if len(verts)>0:
        path = Path(verts, codes)
        patch = patches.PathPatch(path, facecolor='none', lw=5)
        ax.add_patch(patch)
        plt.gca().invert_yaxis() # y values increase as you go down in image
#         plt.show() 
    
    if len(save_path)>0:
        print save_path
        fig.savefig(save_path,bbox_inches='tight')
        # load, resize, then resave
        im = Image.open(save_path)
        im = im.resize((224,224), Image.ANTIALIAS)
        im.save(save_path)          

In [52]:
# path to npz files
path_npz = '/data2/jefan/quickDraw/npz'
path_png = '/data2/jefan/quickDraw/png'

# how many drawings from each category to include
sample_size = 1000

In [53]:
# sorted list of all categories
paths = [os.path.join(path_npz,i) for i in sorted(os.listdir(path_npz))]

In [57]:
# loop through paths and render out
for path in paths:
    print path
    category = path.split('F')[1].split('.')[0]
    x = np.load(path)
    y = x['train']
    for i in np.arange(sample_size):    
        lines = strokes_to_lines(y[i])
        verts,codes = polyline_pathmaker(lines)
        save_path = os.path.join(path_png,category,'{:04d}.png'.format(i))   
        if not os.path.exists(os.path.join(path_png,category)):
            os.makedirs(os.path.join(path_png,category))
        path_renderer(verts, codes,save_path)
        plt.close('all')

